# Imports

In [1]:
from bovw_cam.feature_extractor import FeatureExtractor
from bovw_cam.bovw import BoVW
from bovw_cam.bovw_cam import BoVWCAM
from bovw_cam.classifier_interface import ClassifierInterface

from sklearn import svm
from matplotlib import pyplot as plt

import cv2

ModuleNotFoundError: No module named 'src'

# Feature extraction with SIFT

In [ ]:
feature_extractor = FeatureExtractor()

train, test = feature_extractor.create_train_test_split('/home/arnaldo/Pictures/toy_classification_dataset/train',
                                                        '/home/arnaldo/Pictures/toy_classification_dataset/test')

dictionary = feature_extractor.create_dictionary_split('/home/arnaldo/Pictures/toy_classification_dataset/dict', 0.9)

In [ ]:
train

# Bag-of-Visual-Words

In [ ]:
bovw = BoVW(dictionary, 256, 32)

dict_histograms, dict_metadata = bovw.get_feature_vectors(dictionary)
train_histograms, train_metadata = bovw.get_feature_vectors(train)
test_histograms, test_metadata = bovw.get_feature_vectors(test)

In [ ]:
train_histograms

In [ ]:
train_metadata

# Train SVM

In [ ]:
svm = svm.SVC()

X_train = train_histograms.drop(['image_path', 'target'], axis=1)
y_train = train_histograms['target']

svm.fit(X_train, y_train)

# Classification interface implementation

In [ ]:
from typing import Any


class MyClassifier(ClassifierInterface):
    def __init__(self, model) -> None:
        super().__init__(model)
        self.map_class = {'class_1': 0, 'class_2': 1}

    def predict(self, input_sample: Any) -> int:
        predicted_class = self.model.predict(input_sample)
        return self.map_class[predicted_class[0]]
    
my_classfier = MyClassifier(svm)

# Bag-of-Visual-Words-Class-Activation-Map

In [ ]:
MAX_POOLING_PARAMS = {'kernel_size': 32, 'stride': 8}

bovw_cam = BoVWCAM(my_classfier, dict_histograms)

cam = bovw_cam.get_class_activation_map('/home/arnaldo/Pictures/toy_classification_dataset/test/class_2/1584679085.ac7be78536.jpg', 
                                        test_histograms, 
                                        test_metadata, 
                                        MAX_POOLING_PARAMS)

cam_with_color = cv2.applyColorMap(cam, cv2.COLORMAP_HOT)

plt.imshow(cam_with_color)